In [ ]:
# to import the necessary libraries

import pyActigraphy
import pandas as pd
import glob
import numpy as np
from scipy import ndimage
from pyActigraphy.metrics.metrics import _lmx
from pyActigraphy.sleep.scoring_base import _td_format
from pyActigraphy.analysis import Cosinor

# to read every txt file in the folder

filenames = sorted(glob.glob('*.txt'))


# to read the raw data from each txt file

for f in filenames:
    rawATR = pyActigraphy.io.read_raw_atr(f, period='5 days')
    #rawATR = pyActigraphy.io.read_raw_atr(f)
    

# Just an example of variables we can extract. It calculates the variables and 
# print them in each original file to keep the link with file name

    #roenneberg = rawATR.Roenneberg()
    #roenneberg_thr = rawATR.Roenneberg(threshold=0.25, min_seed_period='10min')
    #aot = rawATR.Roenneberg_AoT()
    #sleepbouts = rawATR.sleep_durations(duration_min='4h', algo='Roenneberg')
    Onset = rawATR.AonT(freq='10min', binarize=True)
    Offset = rawATR.AoffT(freq='10min', binarize=True)
    
    raw = rawATR
    duration = raw.duration()
    
    
    IS = rawATR.IS()
    IV = rawATR.IV()
    RA = rawATR.RA(binarize=True)
    M10 = rawATR.M10(binarize=True)
    L5 = rawATR.L5(binarize=True)
    
    lm5_ts, lm5 = _lmx(rawATR.resampled_data('10min'), '5h', lowest=True)
    lm10_ts, lm10 = _lmx(rawATR.resampled_data('10min'), '10h', lowest=False)
    
    if lm5_ts<pd.Timedelta('24h'):
        lm5_start = lm5_ts 
    else:
        lm5_start = lm5_ts - pd.Timedelta('24h')
        lm5_stop = lm5_start + pd.Timedelta('5h')
    
    if lm10_ts<pd.Timedelta('24h'):
        lm10_start = lm10_ts 
    else:
        lm10_start = lm10_ts - pd.Timedelta('24h')
        lm10_stop = lm10_start + pd.Timedelta('10h')
    
    L5i = _td_format(lm5_start) 
    M10i = _td_format(lm10_start)
    
    
    ADAT = rawATR.ADAT(binarize=True, threshold=4)
    daily_profile1 = rawATR.average_daily_activity(freq='60min', cyclic=True, binarize=True, threshold=4)
    CoG = ndimage.measurements.center_of_mass(daily_profile1.index)
    
    AUC_total = (np.trapz(daily_profile1, axis=0)/2)
    
    cosinor = Cosinor()
    cosinor.fit_initial_params['Period'].value = 1440
    cosinor.fit_initial_params['Period'].vary = False
    results = cosinor.fit(rawATR, verbose=False)
    best_fit = cosinor.best_fit(rawATR, results.params)
    Mesor = results.params['Mesor'].value
    Amplitude = results.params['Amplitude'].value
    Acrophase = results.params['Acrophase'].value
    Acrophase_Cos = Acrophase*229.1831
    
    #SRI = rawATR.SleepRegularityIndex()
    #SMP = rawATR.SleepMidPoint()
    #FS = rawATR.fSoD(start='AonT', period='10h')
    
    
    with open(f, "w") as f:
        
        print(f,duration,IS,IV,RA,M10,L5,M10i,L5i,ADAT,Mesor,Acrophase_Cos,CoG,Onset,Offset, file=f)
        

# a new loop to get the variables from each file and concatenate in a single output file for statistics         
        
with open("output.txt",'w') as outfile:
   for filename in glob.glob('*.txt'):
        with open(filename) as infile:
            for line in infile:
                outfile.write(line) 
        